In [ ]:
%pip install -U adapter-transformers
%pip install optuna

In [1]:
from datasets import load_dataset

scierc_name = 'nsusemiehl/SciERC'
scierc_dataset = load_dataset(scierc_name)
print(scierc_dataset.num_rows)

# acl_arc_name = 'zapsdcn/citation_intent'
# acl_arc_dataset = load_dataset(acl_arc_name)
# print(acl_arc_dataset.num_rows)



Using custom data configuration nsusemiehl--SciERC-f57c64a52b9c80c0
Reusing dataset json (C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
100%|██████████| 3/3 [00:00<00:00, 749.34it/s]

{'train': 3219, 'test': 974, 'validation': 455}


In [2]:
scierc_dataset['train'][0]

{'text': 'The agreement in question involves number in [[ nouns ]] and << reflexive pronouns >> and is syntactic rather than semantic in nature because grammatical number in English , like grammatical gender in languages such as French , is partly arbitrary .',
 'label': 'CONJUNCTION',
 'metadata': [7, 7, 9, 10]}

In [3]:
# Finding the number of labels
import numpy as np
labels = np.unique(np.array(scierc_dataset['train']['label']))
num_of_labels = labels.size

print(labels)
print(num_of_labels)

['COMPARE' 'CONJUNCTION' 'EVALUATE-FOR' 'FEATURE-OF' 'HYPONYM-OF'
 'PART-OF' 'USED-FOR']
7


In [4]:
# encoding the labels
def encode_labels(dataset):
    for i in range(num_of_labels):
        if dataset['label'] == labels[i]:
            dataset['label'] = i
    return dataset

scierc_dataset = scierc_dataset.map(encode_labels)
scierc_dataset['train'][0]

Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-3fa4decd4606a523.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-4d2e52dbe4cdbad6.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-214c6724dd02783d.arrow


{'text': 'The agreement in question involves number in [[ nouns ]] and << reflexive pronouns >> and is syntactic rather than semantic in nature because grammatical number in English , like grammatical gender in languages such as French , is partly arbitrary .',
 'label': 1,
 'metadata': [7, 7, 9, 10]}

In [5]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
scierc_dataset = scierc_dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
scierc_dataset = scierc_dataset.rename_column("label", 'labels')
# Transform to pytorch tensors and only output the required columns
scierc_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-66a8ac042a6f2d30.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-10b38373a51fb787.arrow
Loading cached processed dataset at C:\Users\The Doctor\.cache\huggingface\datasets\json\nsusemiehl--SciERC-f57c64a52b9c80c0\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-500b8b5d9ccb8f86.arrow


In [6]:
from transformers import RobertaConfig
from transformers import RobertaAdapterModel

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=num_of_labels,
)
model = RobertaAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
# Add new adapter
model.add_adapter("sci_erc")
# Add a matching classification head
model.add_classification_head(
    "sci_erc",
    num_labels=num_of_labels,
    id2label={0:'COMPARE', 1:'CONJUNCTION', 2:'EVALUATE-FOR', 
            3:'FEATURE-OF', 4:'HYPONYM-OF', 5:'PART-OF', 6:'USED-FOR'}
)
# Activate the adapter
model.train_adapter("sci_erc")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaAdapterModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# We have to create a function to create a new model for hyperparameter tuning to work. This allows the tuner to create a new model to test on.
def model_init():
    config = RobertaConfig.from_pretrained(
        "roberta-base",
        num_labels=num_of_labels,
    )
    model = RobertaAdapterModel.from_pretrained(
        "roberta-base",
        config=config,
    )
    # Add new adapter
    model.add_adapter("sci_erc")
    # Add a matching classification head
    model.add_classification_head(
        "sci_erc",
        num_labels=num_of_labels,
        id2label={0:'COMPARE', 1:'CONJUNCTION', 2:'EVALUATE-FOR', 
                3:'FEATURE-OF', 4:'HYPONYM-OF', 5:'PART-OF', 6:'USED-FOR'}
    )
    # Activate the adapter
    model.train_adapter("sci_erc")
    return model

In [8]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    logging_steps=100,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
)

# We want to minimize the loss function so we should not include f1.

# Adding Support for Tensorboard, supposedly you don't have to do this, but I find that it doesn't work
from torch.utils.tensorboard import SummaryWriter
from transformers.integrations import TensorBoardCallback
writer = SummaryWriter()
writer = TensorBoardCallback(writer)

# This will throw a warning that we should not have both a model and a model_init, but adaptertrainer does not like it when we don't have a model.

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=scierc_dataset["train"],
    eval_dataset=scierc_dataset["validation"],
    model_init=model_init,
    callbacks=[writer]   
)

C:\Users\The Doctor\.conda\envs\pytorch\lib\site-packages\transformers\trainer.py:327: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(
You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
AdapterTrainerCallback


In [9]:
# We are using optuna for tuning, by default it tries epochs, learning rate, and batch siz, we can try to give it more parameters to search but I feel like these might be the best for now.
# This will by default run a 100 trials.

trainer.hyperparameter_search(
    # hp_space = {Parameters to tune}
)

 98%|█████████▊| 200/204 [06:19<00:08,  2.12s/it]***** Running Evaluation *****
  Num examples = 455
  Batch size = 128


{'loss': 1.8147, 'learning_rate': 1.0166519921667562e-07, 'epoch': 3.92}


                                                 
 98%|█████████▊| 200/204 [06:27<00:08,  2.12s/it]

{'eval_loss': 1.8343709707260132, 'eval_runtime': 7.658, 'eval_samples_per_second': 59.415, 'eval_steps_per_second': 0.522, 'epoch': 3.92}


100%|██████████| 204/204 [06:34<00:00,  2.49s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 204/204 [06:34<00:00,  1.93s/it]
[I 2022-04-20 16:26:46,716] Trial 4 finished with value: 1.8343709707260132 and parameters: {'learning_rate': 5.184925160050457e-06, 'num_train_epochs': 4, 'seed': 27, 'per_device_train_batch_size': 64}. Best is trial 2 with value: 1.3941717147827148.
Trial:
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\The Doctor/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id

{'train_runtime': 394.2735, 'train_samples_per_second': 32.658, 'train_steps_per_second': 0.517, 'train_loss': 1.8578294188368554, 'epoch': 4.0}


loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from cache at C:\Users\The Doctor/.cache\huggingface\transformers\51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaAdapterModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification 

KeyboardInterrupt: 

In [ ]:
trainer.evaluate(scierc_dataset['test'])